In [1]:
from model import GPT
import numpy as np
import torch
from tqdm import tqdm
from copy import deepcopy
import tiktoken
from datasets import load_from_disk

In [2]:
orthogonal_model = GPT.from_pretrained('/home/mfx/huggingface/gpt2-xl')
device = "cuda:3"
state_dict = torch.load("/home/mfx/CLOVer/output/orthogonal/gpt2-xl.pt")
orthogonal_model.load_state_dict(state_dict)
orthogonal_model = orthogonal_model.to(device)
enc = tiktoken.get_encoding("gpt2")
test = load_from_disk("/home/mfx/CLOVer/output/wikitext-2-raw-v1")['test']
all_input_ids = torch.LongTensor(enc.encode("\n\n".join(test["text"]))).unsqueeze(0)
seq_len = all_input_ids.size(1)-1

loading weights from pretrained gpt: gpt2-xl
forcing vocab_size=50257, block_size=1024, bias=True
number of parameters: 1555.97M


/tmp/ipykernel_3308846/4153415480.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/home/mfx/CLOVer/output/orthogonal/gpt2-xl.pt")


In [3]:
def calculating_perplexity(model, datasest):
    max_length = model.config.block_size
    stride = 512
    nll_sum = 0.0
    n_tokens = 0
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = datasest[:, begin_loc:end_loc].to(device)
        target_ids = datasest[:, begin_loc+1:end_loc+1].to(device)
        #target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100
        with torch.no_grad():
            _,loss,_,_ = model(input_ids, targets=target_ids)

            # loss is calculated using CrossEntropyLoss which averages over valid labels
            # N.B. the model only calculates loss over trg_len - 1 labels, because it internally shifts the labels
            # to the left by 1.
            neg_log_likelihood = loss

        # Accumulate the total negative log-likelihood and the total number of tokens
        num_valid_tokens = (target_ids != -100).sum().item()  # number of valid tokens in target_ids
        batch_size = target_ids.size(0)
        num_loss_tokens = num_valid_tokens - batch_size  # subtract batch_size due to internal label shift
        nll_sum += neg_log_likelihood * num_loss_tokens

        n_tokens += num_loss_tokens
        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    avg_nll = nll_sum / n_tokens  # average negative log-likelihood per token
    ppl = torch.exp(avg_nll)
    return avg_nll, ppl
    

In [4]:
calculating_perplexity(orthogonal_model, all_input_ids)

100%|█████████▉| 560/562 [01:04<00:00,  8.63it/s]


(tensor(2.6933, device='cuda:3'), tensor(14.7811, device='cuda:3'))

In [4]:
qk_norm_list = []
vo_norm_list = []
for name, module in orthogonal_model.named_modules():
    if name.endswith("attn"):
        q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
        q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
        k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
        k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
        qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
        qk_norm_list.append(qk_norm)
        
        v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
        v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
        v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
        o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
        vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
        vo_norm_list.append(vo_norm)
qk_norm_list = torch.stack(qk_norm_list)
vo_norm_list = torch.stack(vo_norm_list)

In [5]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in orthogonal_model.named_modules():
        if name.endswith("attn"):
            q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
            q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
            k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
            k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
            qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
            module.q_proj.weight.data[qk_norm<qk_quantile]=0
            module.q_proj.bias.data[qk_norm<qk_quantile]=0
            module.k_proj.weight.data[qk_norm<qk_quantile]=0
            module.k_proj.bias.data[qk_norm<qk_quantile]=0
    print(calculating_perplexity(orthogonal_model, all_input_ids))
    

0.0 tensor(0.2857, device='cuda:3') tensor(0.1108, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.68it/s]


(tensor(2.6933, device='cuda:3'), tensor(14.7811, device='cuda:3'))
0.015625 tensor(0.6729, device='cuda:3') tensor(0.4198, device='cuda:3')


100%|█████████▉| 560/562 [01:05<00:00,  8.57it/s]


(tensor(2.6937, device='cuda:3'), tensor(14.7858, device='cuda:3'))
0.03125 tensor(1.0134, device='cuda:3') tensor(0.5602, device='cuda:3')


...

(tensor(6.0155, device='cuda:3'), tensor(409.7274, device='cuda:3'))
0.953125 tensor(5.7999, device='cuda:3') tensor(3.6818, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.67it/s]


(tensor(6.2179, device='cuda:3'), tensor(501.6692, device='cuda:3'))
0.96875 tensor(6.5356, device='cuda:3') tensor(3.9532, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.67it/s]

(tensor(6.4243, device='cuda:3'), tensor(616.6791, device='cuda:3'))


In [5]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in orthogonal_model.named_modules():
        if name.endswith("attn"):
            v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
            v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
            o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
            vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
            module.v_proj.weight.data[vo_norm<vo_quantile]=0
            module.v_proj.bias.data[vo_norm<vo_quantile]=0
            module.v_proj.weight.data[:,vo_norm<vo_quantile]=0
    print(calculating_perplexity(orthogonal_model, all_input_ids))
    

0.0 tensor(0.2857, device='cuda:3') tensor(0.1108, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.70it/s]


(tensor(2.6933, device='cuda:3'), tensor(14.7811, device='cuda:3'))
0.015625 tensor(0.6729, device='cuda:3') tensor(0.4198, device='cuda:3')


100%|█████████▉| 560/562 [01:05<00:00,  8.59it/s]


(tensor(2.6925, device='cuda:3'), tensor(14.7690, device='cuda:3'))
0.03125 tensor(1.0134, device='cuda:3') tensor(0.5602, device='cuda:3')


...

(tensor(7.6158, device='cuda:3'), tensor(2029.9540, device='cuda:3'))
0.953125 tensor(5.7999, device='cuda:3') tensor(3.6818, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.69it/s]


(tensor(7.6158, device='cuda:3'), tensor(2029.9540, device='cuda:3'))
0.96875 tensor(6.5356, device='cuda:3') tensor(3.9532, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.69it/s]

(tensor(7.6158, device='cuda:3'), tensor(2029.9540, device='cuda:3'))


In [ ]:
for i in range(63):
    ratio = i/64
    qk_quantile = torch.quantile(qk_norm_list, ratio)
    vo_quantile = torch.quantile(vo_norm_list, ratio)
    print(ratio, qk_quantile, vo_quantile)
    for name, module in orthogonal_model.named_modules():
        if name.endswith("attn"):
            q_weight = deepcopy(module.q_proj.weight.data) # (hidden_size, q_in_dim)
            q_bias = deepcopy(module.q_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            q_weight = torch.cat([q_weight, q_bias],dim=1)  # (hidden_size, q_in_dim+1)
            k_weight = deepcopy(module.k_proj.weight.data) # (hidden_size, k_in_dim)
            k_bias = deepcopy(module.k_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            k_weight = torch.cat([k_weight, k_bias],dim=1)  # (hidden_size, k_in_dim+1)
            qk_norm = q_weight.norm(p=2,dim=-1) * k_weight.norm(p=2,dim=-1)
            module.q_proj.weight.data[qk_norm<qk_quantile]=0
            module.q_proj.bias.data[qk_norm<qk_quantile]=0
            module.k_proj.weight.data[qk_norm<qk_quantile]=0
            module.k_proj.bias.data[qk_norm<qk_quantile]=0
            v_weight = deepcopy(module.v_proj.weight.data) # (hidden_size, v_in_dim)
            v_bias = deepcopy(module.v_proj.bias.data).unsqueeze(1) # (hidden_size, 1)
            v_weight = torch.cat([v_weight, v_bias],dim=1)  # (hidden_size, v_in_dim+1)
            o_weight = deepcopy(module.o_proj.weight.data) # (o_out_dim,hidden_size)
            vo_norm = v_weight.norm(p=2,dim=-1) * o_weight.norm(p=2,dim=0)
            module.v_proj.weight.data[vo_norm<vo_quantile]=0
            module.v_proj.bias.data[vo_norm<vo_quantile]=0
            module.v_proj.weight.data[:,vo_norm<vo_quantile]=0
    print(calculating_perplexity(orthogonal_model, all_input_ids))
    

0.0 tensor(0.2857, device='cuda:3') tensor(0.1108, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.69it/s]


(tensor(2.6933, device='cuda:3'), tensor(14.7811, device='cuda:3'))
0.015625 tensor(0.6729, device='cuda:3') tensor(0.4198, device='cuda:3')


100%|█████████▉| 560/562 [01:05<00:00,  8.58it/s]


(tensor(2.6929, device='cuda:3'), tensor(14.7742, device='cuda:3'))
0.03125 tensor(1.0134, device='cuda:3') tensor(0.5602, device='cuda:3')


...

100%|█████████▉| 560/562 [01:05<00:00,  8.61it/s]


(tensor(6.5179, device='cuda:3'), tensor(677.1791, device='cuda:3'))
0.265625 tensor(2.3567, device='cuda:3') tensor(1.3267, device='cuda:3')


100%|█████████▉| 560/562 [01:04<00:00,  8.62it/s]


(tensor(6.4258, device='cuda:3'), tensor(617.5693, device='cuda:3'))
0.28125 tensor(2.3911, device='cuda:3') tensor(1.3595, device='cuda:3')


 40%|████      | 226/562 [00:26<00:38,  8.63it/s]

In [7]:
ids = torch.LongTensor(enc.encode("American identity. I am part of a resistance movement with my peers,")).unsqueeze(0).to("cuda:3")
output = orthogonal_model.generate(idx=ids, max_new_tokens=15)
enc.decode(output[0].tolist())

'American identity. I am part of a resistance movement with my peers,17FA) Vic To beaches\nspeading tostyle with the 21'